# DB evaluation 

In [2]:
import pandas as pd
import json
import os
import time
import itertools
import sys
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
import os
from plotly import figure_factory as ff

In [3]:
import sys
print (sys.version)
print(pd.__version__)

3.6.0 |Anaconda 4.3.1 (x86_64)| (default, Dec 23 2016, 13:19:00) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
0.19.2


## Some statistics 

In [4]:
df = pd.read_json('duplicates_and_original_questions.json',orient= 'records',numpy = False,dtype = True,lines =True)

ValueError: Expected object or value

In [4]:
df.head()

,answer,crawl_date,product,question,source_domain,source_url,uid
0,{'answer_body': ' You need to check it in wind...,2017-04-10T13:16:01.336850,ubuntu,{'question_body': ' This question already has...,superuser.com,https://superuser.com/questions/1020804/list-o...,144994b8-1ddf-11e7-881f-9d7e2576939d
1,{'answer_body': ' If you connect a LAN port on...,2017-04-10T13:17:40.724953,ubuntu,{'question_body': ' This question already has...,superuser.com,https://superuser.com/questions/1101566/can-i-...,4f870080-1ddf-11e7-98aa-9d7e2576939d
2,{'answer_body': ' Unless you're trying to run ...,2017-04-10T13:19:30.889944,ubuntu,{'question_body': ' This question already has...,superuser.com,https://superuser.com/questions/1152621/can-i-...,9130b150-1ddf-11e7-a9f9-9d7e2576939d
3,"{'answer_body': ' See here, here and here. EDI...",2017-04-10T13:21:12.485814,ubuntu,{'question_body': ' This question already has...,superuser.com,https://superuser.com/questions/846275/pes-201...,cdbf251c-1ddf-11e7-8122-9d7e2576939d
4,{'answer_body': ' Check your BIOS/UFI settings...,2017-04-10T13:23:30.594418,ubuntu,{'question_body': ' This question already has...,superuser.com,https://superuser.com/questions/1165109/samsun...,2010d876-1de0-11e7-94f5-9d7e2576939d


### Basic Statistics

In [5]:
number_of_question = df.shape[0]

In [6]:
number_of_duplicat = 0
c = 0
for e in df['question']:
    
    try :
        line = e['question_original_url']
        number_of_duplicat +=1
        c +=1
    except KeyError:
        df.drop(df.index[c], inplace=True)

In [7]:
print('number of questions',number_of_question)
print('number of duplicate questions',number_of_duplicat)

number of questions 1108
number of duplicate questions 1108


In [8]:
df.to_csv(path_or_buf='evaluationWithOnlyDuplicate.csv')

## Preparing the Evaluation

###### Run the script populating_DB and check your solr database http://localhost:8983/solr/#/Evaluation/query

###### For the Evaluation run the script pipeline_to_evaluate.py it will create a pd_dataframe we will analyse there 

## Evaluation

In [9]:
def find_first_match(input_dataFrame,ref_uid,recall_at):
    ''' Return the position of the first match with the uid
    If first match is greater than recall_at the number is recall_at + 1
    If the is no match return recallat + 2
    '''
    try : 
        number = input_dataFrame.uid[input_dataFrame.uid == ref_uid ].index.tolist()[0]
        if number > recall_at :
            number = recall_at + 1

    except IndexError : 
        number = recall_at + 2
    return number


def compute_recall(uid_list,number_of_uid,recall_at) :
    ''' Given a list of uid return the recall
    '''
    count = 0
    for e in uid_list :
        if  e < recall_at + 1 :
            count +=1 
    
    return count/number_of_uid

In [10]:
# Usefull functions to perform basic statistics
def compteNumberOfMiss(l):
    count = 0
    for e in l:
        if e > recall_at :
            count +=1
    return count

def mean(l):
    mean = 0
    for e in l:
        if e > recall_at :
            pass
        else : 
            mean += e
    if len(l)-compteNumberOfMiss(l) != 0 : 
        return mean/(len(l)-compteNumberOfMiss(l))
    else :
        return -1

def SecondOrder(l):
    second = 0
    for e in l:
        if e > recall_at :
            pass
        else : 
            second += e*e
    return second/(len(l)-compteNumberOfMiss(l))

In [23]:
def display_graph(baseline,superuser,ubuntu,neo4j, mode='RECALL'):
    plot([Scatter(x=np.arange(len(baseline)), y=baseline,mode = 'markers',name = 'SOLR'+mode),
      Scatter(x=np.arange(len(superuser)), y=superuser,mode = 'markers',name = 'RNN superuser'+ mode),
      Scatter(x=np.arange(len(ubuntu)), y=ubuntu,mode = 'markers',name = 'RNN RECALL ubuntu'+ mode),
     Scatter(x=np.arange(len(neo4j)), y=neo4j,mode = 'markers',name = 'NEO4J'+ mode)])
    

def display_statistic_table(baseline,superuser,ubuntu,neo4j, mode='RECALL') :
    count = len(baseline)
    if mode == 'BEST RANKING' :
        data_matrix = [['Model', 'Mean', 'Second Order Moment','Number of miss (%)'],
                   ['Baseline '+mode, mean(baseline), SecondOrder(baseline),compteNumberOfMiss(baseline)/count],
                   ['Rnn superuser '+mode, mean(superuser), SecondOrder(superuser),compteNumberOfMiss(superuser)/count],
                   ['Rnn ubuntu '+mode, mean(ubuntu), SecondOrder(ubuntu),compteNumberOfMiss(ubuntu)/count],
                    ['Neo4j '+mode, mean(neo4j), SecondOrder(neo4j),compteNumberOfMiss(neo4j)/count],
                   ]
    if mode == 'RECALL' : 
        data_matrix = 
        

    table = ff.create_table(data_matrix)
    iplot(table, filename='Model Evaluation')

In [28]:
recall_at = 10

# Evaluation 1 : Best Ranking 
ranking_baseline = []
ranking_rnn_ubuntu = []
ranking_rnn_superuser = []
ranking_neo4j = []

# Evaluation 2 : Recall
frecall_baseline = []
frecall_rnn_ubuntu = []
frecall_rnn_superuser = []
frecall_neo4j = []

In [29]:
count = 0
for filename in os.listdir(str(os.getcwd())+'/Output_duplicat/RNN100'):
        count +=1
        uid = filename[:len(filename)-4]
        ## Load the results
 
        df_baseline=pd.read_csv(str(os.getcwd())+'/Output_duplicat/BASELINE/'+filename)
        df_rnn_superuser=pd.read_csv(str(os.getcwd())+'/Output_duplicat/RNN100/'+filename)
        df_rnn_ubuntu=pd.read_csv(str(os.getcwd())+'/Output_duplicat/RNN200/'+filename)
        df_neo4j=pd.read_csv(str(os.getcwd())+'/Output_duplicat/NEO4j/'+filename)
        text_file = open(str(os.getcwd())+'/Output_duplicat/TEXT/'+ filename)
        lines = text_file.readlines()
        
        rank_first_match_baseline = []
        rank_first_match_rnn_superuser =[]
        rank_first_match_rnn_ubuntu =[]
        rank_first_match_neo4j =[]
        if lines != []:    
            ## Remove uid of the question
            lines.remove(uid+'\n')
            
            for i in range(len(lines)):
                ref_uid = lines[i].rstrip()
                
                # BASELINE
                rank_first_match_baseline.append(find_first_match(df_baseline,ref_uid,recall_at))

                # Neo4j
                rank_first_match_neo4j.append(find_first_match(df_neo4j,ref_uid,recall_at))
                
                # RNN superuser
                rank_first_match_rnn_superuser.append(find_first_match(df_rnn_superuser,ref_uid,recall_at))
                
                # RNN ubuntu
                rank_first_match_rnn_ubuntu.append(find_first_match(df_rnn_ubuntu,ref_uid,recall_at))
            
            try : 
                
                # Evaluation 1 : Best Ranking 
                best_ranking_baseline = min(rank_first_match_baseline)
                best_ranking_rnn_superuser = min(rank_first_match_rnn_superuser)
                best_ranking_rnn_ubuntu = min(rank_first_match_rnn_ubuntu)
                best_ranking_neo4j = min(rank_first_match_neo4j)
                
                # Evaluation 2 : Recall
                recall_neo4j = compute_recall(rank_first_match_neo4j,len(lines),recall_at)
                recall_rnn_superuser = compute_recall(rank_first_match_rnn_superuser,len(lines),recall_at)
                recall_rnn_ubuntu = compute_recall(rank_first_match_rnn_ubuntu,len(lines),recall_at)
                recall_baseline = compute_recall(rank_first_match_baseline,len(lines),recall_at)
                
                
            except ValueError :
                # Error I cannot identify
                print('Error')
                best_ranking_baseline = recall_at + 3 
                best_ranking_rnn_superuser = recall_at + 3
                best_ranking_rnn_ubuntu =recall_at + 3
                best_ranking_neo4j = recall_at + 3
                
                recall_neo4j = recall_at +1
                recall_rnn_superuser = recall_at +1
                recall_rnn_ubuntu = recall_at +1
                recall_baseline = recall_at +1
                
            
        ranking_baseline.append(best_ranking_baseline)
        ranking_rnn_superuser.append(best_ranking_rnn_superuser)
        ranking_rnn_ubuntu.append(best_ranking_rnn_ubuntu)
        ranking_neo4j.append(best_ranking_neo4j)
        
        frecall_baseline.append(recall_baseline)
        frecall_rnn_superuser.append(recall_rnn_superuser)
        frecall_rnn_ubuntu.append(recall_rnn_ubuntu)
        frecall_neo4j.append(recall_neo4j)
print('The number of documents is : ', count)

Error
Error
The number of documents is :  179


###### Evaluation 1 : Best ranking

In [27]:
display_graph(ranking_baseline,ranking_rnn_superuser,ranking_rnn_ubuntu,ranking_neo4j, mode='BEST RANKING')

In [30]:
display_statistic_table(ranking_baseline,ranking_rnn_superuser,ranking_rnn_ubuntu,ranking_neo4j, mode='BEST RANKING')

###### Evaluation 2 : Recall

In [26]:
display_graph(frecall_baseline,frecall_rnn_superuser,frecall_rnn_ubuntu,frecall_neo4j, mode='RECALL')

In [25]:
display_statistic_table(frecall_baseline,frecall_rnn_superuser,frecall_rnn_ubuntu,frecall_neo4j, mode='RECALL')